# DAV ENDSEM EXAM
Dipean Dasgupta<br>
202151188

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.utils import resample
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DS1=pd.read_csv('/content/drive/MyDrive/CS312_DA/Bp.csv')
DS2=pd.read_csv('/content/drive/MyDrive/CS312_DA/Islander_data.csv')
DS3=pd.read_csv('/content/drive/MyDrive/CS312_DA/Mental_health.csv')

In [ ]:
DF1=pd.DataFrame(DS1)

**Question1 Answer**

In [ ]:
num_samples = 10000

sample_size = 20

def calculate_sample_mean(sample):
    return np.mean(sample)

sample_means = []
for _ in range(num_samples):
    sample = DF1['Physical_activity'].sample(sample_size, replace=False)
    sample_mean = calculate_sample_mean(sample)
    sample_means.append(sample_mean)

standard_error = np.std(sample_means) / np.sqrt(sample_size)

print(len(sample_means))
print("Standard Error:", standard_error)

10000
Standard Error: 694.4226732457788


**Question2 Answer**

In [ ]:

filt_bp = DF1[DF1['Blood_Pressure_Abnormality'] == 1]

max_salt = filt_bp['salt_content_in_the_diet'].max()
min_salt = filt_bp['salt_content_in_the_diet'].min()
normalized_salt = (filt_bp['salt_content_in_the_diet'] - min_salt) / (max_salt - min_salt)
filt_bp['normalized_salt'] = normalized_salt

np.random.seed(0)
bootstrap_samples = 10000
salt_samples = []

for _ in range(bootstrap_samples):
    sample = resample(filt_bp['normalized_salt'])
    salt_samples.append(sample.mean())

confidence_interval = np.percentile(salt_samples, [2.5, 97.5])
print('95% Confidence Interval for normalized salt content:', confidence_interval)

<ipython-input-21-f439f201b74f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_bp['normalized_salt'] = normalized_salt


95% Confidence Interval for normalized salt content: [0.48453159 0.52014007]


**Question3 Answer**

In [ ]:
DF3=pd.DataFrame(DS3)

In [ ]:
DF3

In [ ]:
DF3.rename(columns={
    'Have you ever sought treatment for a mental health disorder from a mental health professional?': 'professional_help',
    'What is your gender?': 'gender'
}, inplace=True)
gender_mapping = {
    'Female': 'female',
    'female': 'female',
    'F': 'female',
    'f': 'female',
    'Woman': 'female',
    'femalw': 'female',
    'femail': 'female',
    'My sex is female.': 'female',
    'Woman-identified': 'female',
    'Female (cis)': 'female',
    'Female ': 'female',
    'F, cisgender': 'female',
    'Female-ish': 'female',
    'cis female': 'female',
    'cis-Female': 'female',
    'woman': 'female',
    'Transfeminine': 'female',
    'man': 'male',
    'Male': 'male',
    'male': 'male',
    'M': 'male',
    'cis-male': 'male',
    'Mail': 'male',
    'Male (cis)': 'male',
    'Cis male': 'male',
    'Cis-male': 'male',
    'Male ': 'male',
    'Male, cis': 'male',
    'cis male ': 'male',
    'Male-ish': 'male',
    'Non-binary': np.nan,  # We are excluding non-binary entries
    'Genderqueer/non-binary': np.nan,  # We are excluding non-binary entries
    'Non binary': np.nan,  # We are excluding non-binary entries
    'Genderfluid': np.nan,  # We are excluding non-binary entries
    'Agender/genderfluid': np.nan,  # We are excluding non-binary entries
    'Nonbinary': np.nan,  # We are excluding non-binary entries
    'Genderqueer demigirl': 'female',  # We are excluding non-binary entries
    'cis hetero male': np.nan,  # Invalid entry
    'uhhhhhhhhh fem genderqueer?': 'female',  # Invalid entry
    "male (hey this is the tech industry you're talking about)": 'male',  # Invalid entry
    'sometimes': np.nan,  # Invalid entry
    'God King of the Valajar': np.nan,  # Invalid entry
    'Contextual': np.nan,  # Invalid entry
    '\\-': np.nan,  # Invalid entry
    'Trans woman': 'female',  # Invalid entry
    'Cis Male': 'male',  # Fix capitalization
    'Cis male ': 'male' ,
    'Man':'male',
    'male/androgynous ':'male',
    'm':'male',
       'female (cis)':'female', 'female (cisgender)':'female', 'Female (cis) ':'female','trans woman':'female'

}

DF3['gender'] = DF3['gender'].replace(gender_mapping)


mental = DF3[['professional_help', 'gender']]

contingency_table = pd.crosstab(mental['professional_help'], mental['gender'])

print('Contingency Table:')
print(contingency_table)

print('\nNumber of females who sought professional help:', contingency_table.loc[1, 'female'])

Contingency Table:
gender             dude  female  male  nonbinary
professional_help                               
0                     1      59   234          0
1                     0     165   268          1

Number of females who sought professional help: 165


**Question4 Answer**

In [ ]:
chi2, p, dof, expected = chi2_contingency(contingency_table)

print('Chi-square value:', chi2)

Chi-square value: 28.598046261301075


**Question5 Answer**

In [ ]:
observed = contingency_table.loc[1, 'M']
expected = expected[1, contingency_table.columns.get_loc('M')]

# Calculating Pearson Residual
pearson_residual = (observed - expected) / np.sqrt(expected)
print(f"Pearson Residual for male subjects who choose professional help: {pearson_residual:.4f}")

Pearson Residual for male subjects who choose professional help: -1.8656


**Question6 Answer**

In [ ]:
DF2=pd.DataFrame(DS2)

In [ ]:
new_df = DF2[['Drug', 'Diff']]

model = ols('Diff ~ C(Drug)', data=new_df).fit()
anova_results = sm.stats.anova_lm(model, typ=2)

grand_mean = new_df['Diff'].mean()
print(f"Grand mean of the new data frame: {grand_mean:.4f}")
print(anova_results)

Grand mean of the new data frame: 2.9545
                sum_sq     df          F        PR(>F)
C(Drug)    4304.785154    2.0  22.711289  1.359214e-09
Residual  18480.525755  195.0        NaN           NaN


**Question7 Answer**

In [ ]:
grand_degree_of_freedom = anova_results.loc['Residual', 'df']
print(f"Grand degree of freedom: {grand_degree_of_freedom}")

Grand degree of freedom: 195.0


**Question8 Answer**

In [ ]:
mean_sum_square_residual = anova_results.loc['Residual', 'sum_sq'] / anova_results.loc['Residual', 'df']
print(f"Mean Sum Square of Residual: {mean_sum_square_residual:.4f}")

Mean Sum Square of Residual: 94.7719
